In [ ]:
import requests
import json
import pandas as pd
from time import sleep
from datetime import date

### Função de Busca com Parâmetros inseridos pelo usuário

In [ ]:
def search(city, checkin, checkout, adults=1, childrens=0, infants=0): # 3 parâmentros obrigatórios e 3 opcionais

    headers = {
    "X-RapidAPI-Key": "be40a372c4msha287a041c2472e3p1d7451jsn1789a2973a48", # token e host fornecidos pelo API na hora do cadastro
    "X-RapidAPI-Host": "airbnb13.p.rapidapi.com"
    }

    url = "https://airbnb13.p.rapidapi.com/search-location" # url base da API

    querystring = {"location":city,"checkin":checkin,"checkout":checkout,"adults":adults,"children":childrens,"infants":infants,"page":"1","currency":"BRL"} # lista de parâmentros para a pesquisa.

    # Laço para aquisição dos resultados (maxímo de 8 paginas, com 40 resultados por página. limite da API)        
    results = []
    for page in range(1,9):
        querystring["page"] = str(page)
        if page == 6:
            sleep(60) # A API só deixa fazer 5 requests por minutos. No 6º request, precisa esperar pelo menos 1mim para poder fazer mais.
        response = requests.get(url, headers=headers, params=querystring).json()
        if response.get("error") == True or response.get("message") != None:
            print(response.get("message"))
        else:
            results.extend(response["results"])
    # Criação de um DF com o resultado total encontrado
    df = pd.DataFrame(data=results)

    # Algumas informações mostradas ao usuário como uma base
    print(f"{len(results)} results were found.")
    sum = 0
    for ids in results:
        sum += ids["price"]["total"]
    print(f"The avarege price is {querystring['currency']}:{sum/(int(len(results)))}")

    return df


### Inserção das variáveis para a pesquisa no site

#### Testa data CHECKIN

In [ ]:
def checkin():
    atual_date = date.today()
    temp = str(input("Checkin (Format: aaaa/mm/dd) *Required: "))
    checkin = str(temp.split("-")[::-1])
    if checkin < str(atual_date):
        print("Date can not be in the past!")
        checkin()
    return checkin

#### Testa data CHECKOUT

In [ ]:
def checkout(var):
    temp = str(input("Checkout (Format: aaaa/mm/dd) *Required: "))
    checkout = str(temp.split("-")[::-1])
    if checkout <= var:
        print("Checkout must be after checkin")
        checkout()
    return checkout

### Entrada dos parâmetros de pesquisa

In [ ]:
city = str(input("Enter the City *Required: "))
checkin = checkin()
checkout = checkout(checkin)
adults = str(input("Adults (+13y) *Default = 1): "))
childrens = str(input("Childerns (2-12y) *Default = 0): "))
infants = str(input("Infants (U-2y) *Default = 0): "))

df = search(city, checkin, checkout, adults, childrens, infants) # Chamada da função com os parâmentros

### Função Filtro

In [ ]:
df.loc[df.bathrooms==2]